R1 dengan w2v model yg di peroleh dari pretraining model wikipedia indonesia.
yg di process unigram, basis dari paper Yujun

In [1]:
#1. rutin1 import module
import pandas as pd
import os
import sys
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

In [3]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "data/dataset_ekstraksi_r29.xlsx")
df = pd.read_excel(dataset_path)
df["text"] = df["judul"] +". "+ df["isi"]

In [4]:
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary

def preprocess(text):
    '''
    fungsi untuk menghilangkan karakter yg tidak bermakna dan menghilangkan stopword.
    referensi stopword: tala + sastrawi + custom
    '''
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = text.replace('.', '. ')
    text = re.sub('[^a-zA-Z.]', ' ', text)
    text = text.lower()
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.strip()

    stopwords_path = os.path.join(repo_root, "data/all_stop_words.txt")
    with open(stopwords_path, 'r') as f:
        stopwords = [line.strip() for line in f]
    
    dictionary = ArrayDictionary(stopwords)
    str = StopWordRemover(dictionary)
    text = str.remove(text) # 2x cleaning stop word
    text = str.remove(text)
    return text

def preprocess_tokenize(text):
    '''
    fungsi untuk memproses text yg sudah di process di preprocess() menjadi token
    dilakukan 3x preprocessing, karena stopword masih sering lewat kalau hanya 1x
    '''
    text = preprocess(text)
    text = preprocess(text)
    text = preprocess(text)
    tokens = text.split()
    tokens = [token for token in tokens if token]  # remove any empty tokens
    return tokens

In [5]:
def gen_word_embeddings(text_row, w2v_model):
    # Get the word embeddings for each word in the preprocessed text
    word_embeddings = {}
    for word in text_row:
        if word in w2v_model.wv:
            word_embeddings[word] = w2v_model.wv[word]
    return word_embeddings

In [6]:
def construct_weighted_graph(word_embeddings):
    # Calculate the cosine similarity matrix between all pairs of word embeddings
    cosine_similarities = cosine_similarity(list(word_embeddings.values()))
    # Construct a weighted graph representation of the text
    graph = {}
    for i, word_i in enumerate(word_embeddings.keys()):
        graph[word_i] = {}
        for j, word_j in enumerate(word_embeddings.keys()):
            if i != j:
                graph[word_i][word_j] = cosine_similarities[i][j]
    return graph

In [7]:
def textrank(graph, d=0.85, max_iter=100, tol=1e-4):
    # Initialize all node scores to 1
    scores = {node: 1 for node in graph.keys()}
    # Iterate until convergence
    for i in range(max_iter):
        old_scores = dict(scores)
        for node_i, neighbors in graph.items():
            # Calculate the new score for node i
            score_i = 1 - d
            for node_j, weight_ij in neighbors.items():
                score_i += d * weight_ij * scores[node_j] / sum(graph[node_j].values())
            scores[node_i] = score_i
        # Check for convergence
        max_diff = max([abs(old_scores[node] - scores[node]) for node in graph.keys()])
        if max_diff < tol:
            break
    return scores

In [8]:
def extract_keywords(w2v_model, text_row, n=10):
    preprocessed_text = preprocess_tokenize(text_row)
    word_embeddings = gen_word_embeddings(preprocessed_text, w2v_model)
    graph = construct_weighted_graph(word_embeddings)
    scores = textrank(graph)
    sorted_words = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    top_n_keywords = [word for word, score in sorted_words[:n]]
    keyword_df = pd.DataFrame({'keywords': top_n_keywords, 'score': [score for word, score in sorted_words[:n]]})

    return keyword_df

In [9]:
from gensim.models import Word2Vec

pred_w2v_tr_tune = pd.DataFrame()
w2v_path = os.path.join(repo_root, "models/w2v/idwiki_word2vec_100_new_lower.model")
w2v_model = Word2Vec.load(w2v_path)
for i in df.index:
    try:
        print('Processing index', i, end='...! ')
        text_row = df['text'][i]
        unigram = extract_keywords(w2v_model, text_row, n=3).reset_index(drop=True)
        a = pd.DataFrame(unigram.keywords).T.reset_index(drop=True)
        b = pd.DataFrame(unigram.score).T.reset_index(drop=True)

        # add extra empty columns to a and b dataframes if necessary
        if a.shape[1] < 3:
            for i in range(3 - a.shape[1]):
                a[f'col{i+1}'] = ''
                b[f'col{i+1}'] = ''
        unigram = pd.concat([a, b], axis=1)
        if unigram.shape[1] < 6:
            for i in range(6 - unigram.shape[1]):
                unigram[f'col{i+1}'] = ''
        unigram.columns = ['key_1', 'key_2','key_3','score_1', 'score_2','score_3']

        pred_w2v_tr_tune = pd.concat([pred_w2v_tr_tune, unigram], ignore_index=True)
        pred_w2v_tr_tune[['score_1', 'score_2', 'score_3']] = pred_w2v_tr_tune[['score_1', 'score_2', 'score_3']].round(3)
        print('Done')
    except NameError as e:
        print('Error on index', i)
        break
        
pred_w2v_tr_tune

Processing index 0...! Done
Processing index 1...! Done
Processing index 2...! Done
Processing index 3...! Done
Processing index 4...! Done
Processing index 5...! Done
Processing index 6...! Done
Processing index 7...! Done
Processing index 8...! Done
Processing index 9...! Done
Processing index 10...! Done
Processing index 11...! Done
Processing index 12...! Done
Processing index 13...! Done
Processing index 14...! Done
Processing index 15...! Done
Processing index 16...! Done
Processing index 17...! Done
Processing index 18...! Done
Processing index 19...! Done
Processing index 20...! Done
Processing index 21...! Done
Processing index 22...! Done
Processing index 23...! Done
Processing index 24...! Done
Processing index 25...! Done
Processing index 26...! Done
Processing index 27...! Done
Processing index 28...! Done
Processing index 29...! Done
Processing index 30...! Done
Processing index 31...! Done
Processing index 32...! Done
Processing index 33...! Done
Processing index 34...! 

Processing index 276...! Done
Processing index 277...! Done
Processing index 278...! Done
Processing index 279...! Done
Processing index 280...! Done
Processing index 281...! Done
Processing index 282...! Done
Processing index 283...! Done
Processing index 284...! Done
Processing index 285...! Done
Processing index 286...! Done
Processing index 287...! Done
Processing index 288...! Done
Processing index 289...! Done
Processing index 290...! Done
Processing index 291...! Done
Processing index 292...! Done
Processing index 293...! Done
Processing index 294...! Done
Processing index 295...! Done
Processing index 296...! Done
Processing index 297...! Done
Processing index 298...! Done
Processing index 299...! Done
Processing index 300...! Done
Processing index 301...! Done
Processing index 302...! Done
Processing index 303...! Done
Processing index 304...! Done
Processing index 305...! Done
Processing index 306...! Done
Processing index 307...! Done
Processing index 308...! Done
Processing

Processing index 550...! Done
Processing index 551...! Done
Processing index 552...! Done
Processing index 553...! Done
Processing index 554...! Done
Processing index 555...! Done
Processing index 556...! Done
Processing index 557...! Done
Processing index 558...! Done
Processing index 559...! Done
Processing index 560...! Done
Processing index 561...! Done
Processing index 562...! Done
Processing index 563...! Done
Processing index 564...! Done
Processing index 565...! Done
Processing index 566...! Done
Processing index 567...! Done
Processing index 568...! Done
Processing index 569...! Done
Processing index 570...! Done
Processing index 571...! Done
Processing index 572...! Done
Processing index 573...! Done
Processing index 574...! Done
Processing index 575...! Done
Processing index 576...! Done
Processing index 577...! Done
Processing index 578...! Done
Processing index 579...! Done
Processing index 580...! Done
Processing index 581...! Done
Processing index 582...! Done
Processing

Processing index 823...! Done
Processing index 824...! Done
Processing index 825...! Done
Processing index 826...! Done
Processing index 827...! Done
Processing index 828...! Done
Processing index 829...! Done
Processing index 830...! Done
Processing index 831...! Done
Processing index 832...! Done
Processing index 833...! Done
Processing index 834...! Done
Processing index 835...! Done
Processing index 836...! Done
Processing index 837...! Done
Processing index 838...! Done
Processing index 839...! Done
Processing index 840...! Done
Processing index 841...! Done
Processing index 842...! Done
Processing index 843...! Done
Processing index 844...! Done
Processing index 845...! Done
Processing index 846...! Done
Processing index 847...! Done
Processing index 848...! Done
Processing index 849...! Done
Processing index 850...! Done
Processing index 851...! Done
Processing index 852...! Done
Processing index 853...! Done
Processing index 854...! Done
Processing index 855...! Done
Processing

,key_1,key_2,key_3,score_1,score_2,score_3
0,penilaian,prosedur,teknis,1.699000e+00,1.597000e+00,1.426000e+00
1,facilities,biru,processing,6.816661e+121,5.647531e+121,1.303964e+121
2,specification,table,services,1.717000e+00,1.713000e+00,1.630000e+00
3,pengadaan,inspeksi,control,2.066000e+00,1.873000e+00,1.733000e+00
4,iwan,bp,hamzah,3.793000e+00,2.629000e+00,2.614000e+00
...,...,...,...,...,...,...
995,copy,scanned,elektronik,1.424374e+56,8.232071e+55,6.298397e+55
996,dukungan,persetujuan,mobilisasi,1.788000e+00,1.787000e+00,1.521000e+00
997,memberitahukan,pemberitahuan,pendukungnya,1.030396e+52,1.020878e+52,1.010731e+52
998,kesepakatan,asli,persyaratan,1.810808e+04,1.775956e+04,1.545062e+04


EVALUASI

In [10]:
from utils import eval

targets = df[["k1", "k2", "k3","k4", "k5", "k6","k7"]].values.tolist()
df_targets = pd.DataFrame(targets)

In [11]:
# Evaluation TextRank
predict_w2v_tr_list = pred_w2v_tr_tune[['key_1','key_2','key_3']].values.tolist()
eval_w2v_textrank = eval(predict_w2v_tr_list, targets, True).round(3)
eval_w2v_textrank.columns = ['key_1', 'key_2','key_3','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_w2v_textrank = eval_w2v_textrank[['key_1', 'key_2','key_3', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
eval_w2v_textrank.head(3)

,key_1,key_2,key_3,flex_recall,flex_prec
0,no_match,full_match,no_match,0.143,0.333
1,no_match,no_match,no_match,0.000,0.000
2,no_match,no_match,partial_match,0.143,0.333


In [12]:
# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
w2v_textrank_recall = eval_w2v_textrank['flex_recall'].mean()
w2v_textrank_prec = eval_w2v_textrank['flex_prec'].mean()
w2v_textrank_f1 = 2 * (w2v_textrank_prec * w2v_textrank_recall) / (w2v_textrank_prec + w2v_textrank_recall)

# Create a DataFrame with the scores
summary = pd.DataFrame({'textrank': [w2v_textrank_recall, w2v_textrank_prec, w2v_textrank_f1]}, index=['recall', 'precision', 'F1'])
summary = summary.round(3)
summary

,textrank
recall,0.055
precision,0.128
F1,0.077


In [13]:
# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_w2v_textrank = pd.concat([pred_w2v_tr_tune, df_targets, eval_w2v_textrank], axis=1)
predict_w2v_textrank.head(3)

,key_1,key_2,key_3,score_1,score_2,score_3,0,1,2,3,4,5,6,key_1,key_2,key_3,flex_recall,flex_prec
0,penilaian,prosedur,teknis,1.699000e+00,1.597000e+00,1.426000e+00,persetujuan tertulis,prosedur,usulan,pengganti,NaN,NaN,NaN,no_match,full_match,no_match,0.143,0.333
1,facilities,biru,processing,6.816661e+121,5.647531e+121,1.303964e+121,template document,exhibit c,acuan,pengelolaan,dokumen,NaN,NaN,no_match,no_match,no_match,0.000,0.000
2,specification,table,services,1.717000e+00,1.713000e+00,1.630000e+00,ruang kantor,change inquiry,lingkup kerja,akomodasi,services for company,exhibit a,NaN,no_match,no_match,partial_match,0.143,0.333


In [14]:
# Write predictions to excel file
from utils import write_excel

sheet_name = 'w2v_ia_textrank'
output_file = 'w2v_ia_textrank.xlsx'
write_excel(predict_w2v_textrank, sheet_name, output_file)